In [2]:
!pip install emoji 

  Created wheel for emoji: filename=emoji-0.6.0-py3-none-any.whl size=49720 sha256=36ffa5100f5e2b0618a69c3976d9acb553480ac6300567feab6f2b16f873ae82
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\4e\bf\6b\2e22b3708d14bf6384f862db539b044d6931bd6b14ad3c9adc
Successfully built emoji


In [1]:
import numpy as np
from emo_utils import *
import emoji
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
X_train, Y_train = read_csv('emoji_ukr.csv')

In [3]:
for idx in range(10):
    print(X_train[idx], label_to_emoji(Y_train[idx]))

Французький макарон такий смачний 🍴
робота жахлива 😞
я засмучений 😞
кинути м'яч ⚾
Гарний жарт 😄
яка ваша улюблена гра в бейсбол ⚾
Я готувала м’ясо 🍴
припиніть возитися 😞
Я хочу китайську їжу 🍴
Давайте підемо грати в бейсбол ⚾


In [4]:
Y_oh_train = convert_to_one_hot(Y_train, C = 5)

In [5]:
idx = 50
print(f"Sentence '{X_train[idx]}' has label index {Y_train[idx]}, which is emoji {label_to_emoji(Y_train[idx])}", )
print(f"Label index {Y_train[idx]} in one-hot encoding format is {Y_oh_train[idx]}")

Sentence 'він зробив дивовижну роботу' has label index 2, which is emoji 😄
Label index 2 in one-hot encoding format is [0. 0. 1. 0. 0.]


In [6]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('fiction.lowercased.tokenized.glove.300d')

In [7]:
word = "батьківщина"
idx = 2898
print("the index of", word, "in the vocabulary is", word_to_index[word])
print("the", str(idx) + "th word in the vocabulary is", index_to_word[idx])

the index of батьківщина in the vocabulary is 2733
the 2898th word in the vocabulary is безбожно


In [8]:
def sentence_to_avg(sentence, word_to_vec_map):

    words = sentence.lower().split()
    avg = np.zeros(word_to_vec_map['привіт'].shape)
    
    total = 0
    for w in words:
        try:
            total += word_to_vec_map[w]
        except:
            total += avg
    avg = total / len(words)
        
    return avg

In [9]:
avg = sentence_to_avg("Я люблю свою неньку", word_to_vec_map)
print("avg = \n", avg)

avg = 
 [-6.8779000e-02  1.9953000e-02 -2.9009950e-01  8.7509500e-02
  1.4155000e-02  2.7551550e-01  7.7740250e-02  6.6722750e-02
  1.4309525e-01 -1.6189925e-01 -3.7249525e-01 -1.2387125e-01
 -1.7837375e-01  1.0524475e+00  2.8254650e-01 -7.0859250e-02
 -1.0817500e-02 -2.9021650e-01  3.7671000e-02  2.6803300e-01
 -1.4620375e-01  2.6981500e-01 -1.7425450e-01 -1.5196750e-01
 -2.6725900e-01  3.0173000e-02 -3.8939900e-01 -8.9320000e-03
 -4.0813500e-01 -1.4401500e-01  3.5114375e-01 -3.3192025e-01
 -4.5284275e-01  2.2350500e-01  2.5134125e-01  3.1897275e-01
 -1.1154250e-01 -2.9278350e-01  1.1589250e-02  5.4871825e-01
 -6.3967175e-01  1.6656225e-01 -3.6987075e-01  1.6039700e-01
  1.9606150e-01 -3.0023325e-01  7.7144250e-02 -6.4164000e-02
  3.8528000e-02  8.9644750e-02 -1.9821750e-02  2.6080975e-01
 -9.0121000e-02  2.7288600e-01 -6.8264300e-01  1.1704150e-01
  2.0684475e-01 -3.4356425e-01  7.0288750e-02 -5.5021000e-02
 -3.6088750e-02 -6.4270750e-02 -2.2802775e-01  1.6983375e-01
  1.1958700e-01 

In [10]:
def model(X, Y, word_to_vec_map, learning_rate = 0.01, num_iterations = 400):
    
    np.random.seed(1)

    m = Y.shape[0]                          # number of training examples
    n_y = 5                                 # number of classes  
    n_h = 300                                # dimensions of the GloVe vectors 
    
    W = np.random.randn(n_y, n_h) / np.sqrt(n_h)
    b = np.zeros((n_y,))
    
    Y_oh = convert_to_one_hot(Y, C = n_y) 
    
    for t in range(num_iterations): 
        for i in range(m):         
            
            avg = sentence_to_avg(X[i], word_to_vec_map)

           
            z = np.dot(W, avg) + b
            a = softmax(z)

            cost = -np.sum(Y_oh[i] * np.log(a))
            
            # Compute gradients 
            dz = a - Y_oh[i]
            dW = np.dot(dz.reshape(n_y,1), avg.reshape(1, n_h))
            db = dz

            W = W - learning_rate * dW
            b = b - learning_rate * db
        
        if t % 100 == 0:
            print("Epoch: " + str(t) + " --- cost = " + str(cost))
            pred = predict(X, Y, W, b, word_to_vec_map) #predict is defined in emo_utils.py

    return pred, W, b

In [11]:
pred, W, b = model(X_train, Y_train, word_to_vec_map)
print(pred)

Epoch: 0 --- cost = 1.1038890515158137
Accuracy: 0.3989071038251366
Epoch: 100 --- cost = 0.40175629021942433
Accuracy: 0.9617486338797814
Epoch: 200 --- cost = 0.26212792966964615
Accuracy: 0.9836065573770492
Epoch: 300 --- cost = 0.18762609102630362
Accuracy: 0.9890710382513661
[[4.]
 [3.]
 [3.]
 [1.]
 [2.]
 [1.]
 [4.]
 [2.]
 [4.]
 [1.]
 [3.]
 [3.]
 [2.]
 [2.]
 [4.]
 [3.]
 [2.]
 [3.]
 [3.]
 [1.]
 [3.]
 [2.]
 [2.]
 [2.]
 [0.]
 [1.]
 [0.]
 [4.]
 [2.]
 [0.]
 [2.]
 [0.]
 [0.]
 [3.]
 [4.]
 [0.]
 [2.]
 [1.]
 [3.]
 [1.]
 [0.]
 [4.]
 [0.]
 [3.]
 [0.]
 [4.]
 [2.]
 [3.]
 [4.]
 [2.]
 [2.]
 [3.]
 [0.]
 [2.]
 [2.]
 [3.]
 [2.]
 [3.]
 [2.]
 [2.]
 [3.]
 [3.]
 [0.]
 [2.]
 [3.]
 [0.]
 [2.]
 [0.]
 [0.]
 [2.]
 [3.]
 [2.]
 [4.]
 [1.]
 [3.]
 [3.]
 [0.]
 [0.]
 [3.]
 [2.]
 [0.]
 [3.]
 [0.]
 [2.]
 [2.]
 [4.]
 [2.]
 [2.]
 [0.]
 [0.]
 [2.]
 [3.]
 [0.]
 [4.]
 [2.]
 [1.]
 [2.]
 [3.]
 [3.]
 [2.]
 [3.]
 [0.]
 [3.]
 [0.]
 [2.]
 [0.]
 [2.]
 [3.]
 [4.]
 [3.]
 [1.]
 [3.]
 [4.]
 [3.]
 [2.]
 [3.]
 [3.]
 [3.]
 [1.]
 [4.]

In [12]:
print("Training set:")
pred_train = predict(X_train, Y_train, W, b, word_to_vec_map)

Training set:
Accuracy: 0.994535519125683


In [13]:
X_my_sentences = np.array(["звучить так смішно", "я хочу їсти", "він кохає її", "оце веселий жарт", "пішли грати футбол", "тепер я сумний", "Тарас смішний хлопчина"])
Y_my_labels = np.array([[0], [0], [2], [1], [4],[3]])

pred = predict(X_my_sentences, Y_my_labels , W, b, word_to_vec_map)
print_predictions(X_my_sentences, pred)

Accuracy: 0.0

звучить так смішно 😄
я хочу їсти 🍴
він кохає її ❤️
оце веселий жарт 😄
пішли грати футбол ⚾
тепер я сумний 😞
Тарас смішний хлопчина 😄


# LSTM

In [14]:
import numpy as np
np.random.seed(0)
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
np.random.seed(1)

Using TensorFlow backend.
C:\Users\User\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\User\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\User\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\User\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:529

In [15]:
for idx, val in enumerate(["Я", "люблю", "кодити"]):
    print(idx,val)

0 Я
1 люблю
2 кодити


In [16]:
def sentences_to_indices(X, word_to_index, max_len):
    
    
    m = X.shape[0]                                  
    
    X_indices = np.zeros((m, max_len))
    
    for i in range(m):                              
        
        sentence_words = X[i].lower().split()
        
        j = 0
        
        for w in sentence_words:
            try:
                X_indices[i, j] = word_to_index[w]
                j = j + 1
            except:
                continue
                
    return X_indices

In [17]:
def pretrained_embedding_layer(word_to_vec_map, word_to_index):
      
    vocab_len = len(word_to_index) + 1                 
    emb_dim = word_to_vec_map["огірок"].shape[0]      
    
    emb_matrix = np.zeros((vocab_len, emb_dim))
    
    for word, idx in word_to_index.items():
        emb_matrix[idx, :] = word_to_vec_map[word]

   
    embedding_layer = Embedding(vocab_len, emb_dim, trainable = False)

    embedding_layer.build((None,)) 
    
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [19]:
maxLen = len(max(X_train, key=len).split())
model = Emojify_V2((maxLen,), word_to_vec_map, word_to_index)
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 7)                 0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 7, 300)            33109500  
_________________________________________________________________
lstm_1 (LSTM)                (None, 7, 128)            219648    
_________________________________________________________________
dropout_1 (Dropout)          (None, 7, 128)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_____________________________________________________

In [20]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 

In [21]:
X_train_indices = sentences_to_indices(X_train, word_to_index, maxLen)
Y_train_oh = convert_to_one_hot(Y_train, C = 5)

In [23]:
model.fit(X_train_indices, Y_train_oh, epochs = 50, batch_size = 32, shuffle=True)

Instructions for updating:
Use tf.cast instead.
Epoch 1/50
183/183 [==============================] - 1s 7ms/step - loss: 1.5779 - accuracy: 0.2350
Epoch 2/50
183/183 [==============================] - 0s 513us/step - loss: 1.4658 - accuracy: 0.3825
Epoch 3/50
183/183 [==============================] - 0s 513us/step - loss: 1.4193 - accuracy: 0.4208
Epoch 4/50
183/183 [==============================] - 0s 524us/step - loss: 1.3083 - accuracy: 0.5027
Epoch 5/50
183/183 [==============================] - 0s 546us/step - loss: 1.1785 - accuracy: 0.5792
Epoch 6/50
183/183 [==============================] - 0s 502us/step - loss: 1.0244 - accuracy: 0.6284
Epoch 7/50
183/183 [==============================] - 0s 502us/step - loss: 0.9149 - accuracy: 0.6776
Epoch 8/50
183/183 [==============================] - 0s 524us/step - loss: 0.7455 - accuracy: 0.7377
Epoch 9/50
183/183 [==============================] - 0s 677us/step - loss: 0.6245 - accuracy: 0.7596
Epoch 10/50
183/183 [===============

In [24]:
X_test_indices = sentences_to_indices(X_test, word_to_index, max_len = maxLen)
Y_test_oh = convert_to_one_hot(Y_test, C = 5)
loss, acc = model.evaluate(X_test_indices, Y_test_oh)
print()
print("Test accuracy = ", acc)

NameError: name 'X_test' is not defined

In [25]:
X_my_sentences = np.array(["звучить так смішно", "я хочу їсти", "він кохає її", "оце веселий жарт", "пішли грати футбол", "тепер я сумний", "Тарас смішний хлопчина"])
Y_my_labels = np.array([[0], [0], [2], [1], [4],[3]])

print_predictions(X_my_sentences, pred)


звучить так смішно 😄
я хочу їсти 🍴
він кохає її ❤️
оце веселий жарт 😄
пішли грати футбол ⚾
тепер я сумний 😞
Тарас смішний хлопчина 😄


In [26]:
X_my_sentences = np.array(["звучить так смішно", "я хочу їсти", "він кохає її", "оце веселий жарт", "пішли грати футбол", "тепер я сумний", "Тарас смішний хлопчина"])
Y_my_labels = np.array([[0], [0], [2], [1], [4],[3]])

print_predictions(X_my_sentences, pred)


звучить так смішно 😄
я хочу їсти 🍴
він кохає її ❤️
оце веселий жарт 😄
пішли грати футбол ⚾
тепер я сумний 😞
Тарас смішний хлопчина 😄


In [36]:
x_test = np.array(['пішли грати баскетбол'])
X_test_indices = sentences_to_indices(x_test, word_to_index, maxLen)
print(x_test[0] +' '+  label_to_emoji(np.argmax(model.predict(X_test_indices))))

пішли грати баскетбол 😄
